In [0]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType

# ✅ Definir el esquema del campo metadata
metadata_schema = StructType([
    StructField("button_id", StringType(), True),
    StructField("page", StringType(), True)
])

# ✅ Leer desde Bronze
df_bronze = spark.table("users_events.bronze.users_events_bronze")
display(df_bronze)

# ✅ Parsear la columna metadata
df_silver = df_bronze.withColumn(
    "metadata_parsed", from_json("metadata", metadata_schema)
).select(
    "user_id",
    "event_timestamp",
    "event_type",
    "session_id",
    "device",
    col("metadata_parsed.button_id").alias("button_id"),
    col("metadata_parsed.page").alias("page")
)

# ✅ Ver datos transformados
display(df_silver)

# ✅ Cambiar al esquema silver
spark.sql("USE CATALOG users_events")
spark.sql("USE SCHEMA silver")

# ✅ Guardar tabla limpia
df_silver.write.mode("overwrite").saveAsTable("users_events_silver")

print("✅ Datos guardados en users_events.silver.users_events_silver")

